# Ejercicio de programacion 1: Regresión lineal

## Introducción

En este ejericio usted implementará regresión lineal con datos. Antes de comenzar le recomendamos
ver los videos, enlaces y notas de clase sobre el tema. Por el momento, lo que neceista
para los problemas de este ejercicio es:

* Algebra lineal: Problemas de mínimos cuadrados. Ecuaciones normales
* Saber obenter el gradiente de una funcion multivariada
* Las recursiones para llegar al minimo son del tipo
$$ \theta_k = \theta_{k-1} - \alpha \nabla J(\theta_{k-1})$$
donde $\theta_k$ es el valor actual, $\theta_{k-1}$ es el valor previo, $\alpha$ es el paso
(este se explica más abjo, y $J$ es la función objetivo. 

Recuerde que necesita importar las librería ```numpy``` y ```matplotlib```.



In [1]:
# para manejar "paths"
import os
import math




# numerical analysis library
import numpy as np

# para hacer plots
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # needed to plot 3-D surfaces


# para incluir los plots dentro del cuaderno y no en ventana separada.
%matplotlib inline

## Debugging

Here are some things to keep in mind throughout this exercise:
Tenga en cuenta para este ejercicio

- Los indices en Python comienzan en 0

- No use listas ni tuplas. User numpy arrays.

- Recuerde las dimensiones de las matrices. Python es estricto. Por ejemplo recuerde que en un producto el numero de columnas de la primera matriz debe coincidir con el numero de filas de la segunda.


<a id="section1"></a>
## 1 Simple python and `numpy` function

Este es un ejercicio de calentamiento en Python

```python
A = np.eye(5)
```
<a id="warmUpExercise"></a>

In [ ]:
def warmUpExercise():
    """
    Ejemplo para calcular la funcion identica. 
    
    Returna
    -------
    A : array
        La matriz identidad  5x5 
    
    Instructiones
    ------------
    Returne la matriz identidad   5x5 
    """    
    # ======== YOUR CODE HERE ======
    A = np.eye(5)   # modify this line
    
    # ==============================
    return A

The previous cell only defines the function `warmUpExercise`. We can now run it by executing the following cell to see its output. You should see output similar to the following:

```python
array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])
```

In [ ]:
warmUpExercise()

### 1.1 Como se califica

Apenas corra el ejercicio se revisa si corrió correctamente. 


## 2 Regresión lineal en una variable

Implemente regresión lineal en una variable para predecir las ganancias de un camión que transporta comida. 
Supgonga que el CEO de una cadena de restaurantes piensa abrir sedes en ciudades diferentes. La cadena tiene
camiones y datos de ganancias y población por ciudad. Usted quiere ayudarle al negocio por medio de los datos
para elegir la próxima ciudad de expansión.


Los datos se encuentran en el directorio ```Data``` el cual contiene el archivo ```ex1data1.txt``` . 
La primera columna tiene la población de la ciudad y la segunda las ganancias en múltiplos de 10.000$.
Valores negativos indican pérdida. 

Las siguientes líneas indican como cargar los datos. 

In [2]:
# Lea datos separados por comas ","
data = np.loadtxt(os.path.join('Data', 'ex1data1.txt'), delimiter=',')
X, y = data[:, 0], data[:, 1]

m = y.size  # número de muestras

### 2.1 Graficación de los datos

Antes de comenzar la tarea es útil entender los datos mediante visualización. Para estos datos usted puede usar un "scatter plot" dato que solo posee una cantidad de puntos con coordenadas de ganancia y población. La mayoría de los problemas que encontrás en
la práctica son de más de dos dimensiones y no se pueden visualizar con gráficos 2D. Hay muchas librerías en Python para
visualizar datos. Por ejemplo [este blog](https://blog.modeanalytics.com/python-data-visualization-libraries/)
muestra algunas de las más populares. 

En este curso nosotros usamos exclusivamente ```matplotlib``` , que es una de las librerías científicas más populares.



complete el siguiente código (en la celda a continuación. Algunas líneas se muestran a continuación.

```python
    pyplot.plot(x, y, 'ro', ms=10, mec='k')
    pyplot.ylabel('Profit in $10,000')
    pyplot.xlabel('Population of City in 10,000s')
```

In [ ]:
def plotData(x, y):
    """
    Se grafican los puntos con coordenadas "x" y "y". 
    
    
    
    Parametros
    ----------
    x : array
        coordenadas en el eje x

    y : array
        coordenadas del eje y
        Data point values for y-axis. Note x and y should have the same size.
    
    Instructions
    ------------
    Grafique los datos de entrenamiento usando las funciones "figure" y "plot".
    Los ejes deben ser etiquetados como indica la ayuda arriba. Las coordenadas
    "x" y "y" son argumentos de la función, como se indica arriba.
    
    
    Ayuda:
    ----
    Puede usar la opción de marcadores 'ro' que indica círculos rojos.
    Puede controlar el tamaño de los círculos mediante el parámetro ms=10,
    donde "ms" significa "marker size". Usted también puede usar 
    You can use the 'ro' option with plot to have the markers
    appear as red circles. Furthermore, you can make the markers larger by
    using plot(..., 'ro', ms=10), where `ms` refers to marker size. You 
    can also set the marker edge color using the `mec` property.
    """
    fig = pyplot.figure()  # open a new figure
    
    # ====================== YOUR CODE HERE ======================= 
    pyplot.plot(x, y, 'ro', ms=10, mec='k')
    pyplot.ylabel('Profit in $10,000')
    pyplot.xlabel('Population of City in 10,000s')

    fig.show()
    # =============================================================


Corra el programa vara visualizar los datos acá. Debe obtener la próxima figura.


![](Figures/dataset1.png)

Ejecute la próxima instrucción.

In [ ]:
plotData(X, y)

<a id="section2"></a>
### 2.2 Descenso con Gradiente.
En esta parte usted debe ajustar una regresión lineal para obtener los parámetros $\theta$ usando
descenso con gradiente.


#### 2.2.1 Ecuaciones correspondientes

El objeto de la regresión lineal consiste en la minimización de la función de costo
$$ J(\theta) = \frac{1}{2m} \sum_{i=1}^m \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2$$

donde la hipótesis $h_\theta(x)$ consiste en el modelo lineal
$$ h_\theta(x) = \theta^Tx = \theta_0 + \theta_1 x_1$$

Recuerde que sus parámetros don los valores $\theta_j$. Estos son los
valores que minimizan la función de costo $J(\theta)$. Una forma de hacer esto
es mediante el método de descenso de gradiente, donde cada iteración actualiza el 
valor de $\theta$.


$$ \theta_j = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^m \left( h_\theta(x^{(i)}) - y^{(i)}\right)x_j^{(i)} \qquad \text{acutalice simultáneamente } \theta_j \text{ para todos los } j$$

Con cada paso del descenso de gradinte sus parámetros $\theta_j$ se acercan al valor óptimo que obtiene
el valor más bajo de la función de costo J($\theta$).

<div class="alert alert-block alert-warning">
**Nota de implementación:** Almacenamos cada fila de la matriz $X$ usando ``ǹumpy```. Tenga en
cuenta que el intercepto es ($\theta_0$), y agregamos una columna adicional a $X$ de unos. Esto
nos permite asumir que $\theta_0$ es un atributo adicional.
</div>


#### 2.2.2 Implementación

Ya tenemos las ecuaciones de la regresión lineal. En la siguiente celda, adicionamos una dimensión más a nuestros datos para acomodar $\theta_0$ como el término del intercepto. Por favor NO ejecute
esta celda más de una vez. 


In [3]:
# Add a column of ones to X. The numpy function stack joins arrays along a given axis. 
# The first axis (axis=0) refers to rows (training examples) 
# and second axis (axis=1) refers to columns (features).
X = np.stack([np.ones(m), X], axis=1)

<a id="section2"></a>
#### 2.2.3 Cómputo de la función de costo $J(\theta)$
A medida que desciende con el gradiente se minimiza la función de costo $J(\theta)$, se debe
monitorear la convergencia mediante el cómputo de la función de costo. En esta sección usted
implementará una función para calcular $J(\theta)$ y verificar la convergencia del descenso con gradiente.

Su próxima tarea consiste en completar el código de la función ```computeCost```  la cual evalúa
$J(\theta)$. Recuerde que las variables $X$  y $y$ no son valores escalares. $X$ es la matriz, cuyas
filas representan las muestras para entrenar el sistema y $y$ es el vector que representa la etiqueta para cada fila de $X$
<a id="computeCost"></a>

In [4]:
def computeCost(X, y, theta):
    """
    Compute el costo para la regresión lineal usando theta como parámetro.
    
    Parameters
    ----------
    X : array
        Los datos de entrada con dimensiones (m x n+1), donde m es el número
        de muestras y n es el número de atributos (features). Asumioms que el vector
        de unos ya fue agregado a la matriz de datos X. Por esto las n+1 columnas.
        
    y : array
        Los valores de las etiquetas para cada muestra de entrada. Es un vector de
        dimension m. 
        
    theta : array
        Estos son los parámetros de aprendizaje. Es un vector de tamaño n+1.
        
    Returns
    -------
    J : float
        El valor de costo.
    
    Instructiones
    ------------
    Calcule el valor de costo para un valor particular de theta.
    Llame J a la función de costo.
    """
    
    # inicialice con valores útiles
    m = y.size  # número de muestras
    
    # debe retornar el valor de J correctamente
    J = 0
    
    # ====================== YOUR CODE HERE =====================
    #Sabiendo que h=X*theta (producto de matriz con vector), se calcula la hipotesis para cada elemento como. 
    h = np.dot(X, theta)
    #Se programa la funcion de costo como la suma de los errores al cuadrado entre la hipotesis y el valor real, las cuales se calculan para todos los x_i y y_i al mismo tiempo con la funcion np.sum y luego se multiplica este vector por 1/2m ya que es una constante
    J = (1/(2 * m)) * np.sum(np.square(h - y))
    # ===========================================================
    return J

Una vez completada la función, debe correr ```computeCost``` dos veces usando
dos inicializaciones de $\theta$. Debe obtener el costo impreso en la pantalla.

In [5]:
J = computeCost(X, y, theta=np.array([0.0, 0.0]))
print('Con theta = [0, 0] \nCosto calculado = %.2f' % J)
print('Valor esperado  (aproximadamente) 32.07\n')

# further testing of the cost function
J = computeCost(X, y, theta=np.array([-1, 2]))
print('With theta = [-1, 2]\nCosto calculado = %.2f' % J)
print('Valor esperado aproximadamente) 54.24')

Con theta = [0, 0] 
Costo calculado = 32.07
Valor esperado  (aproximadamente) 32.07

With theta = [-1, 2]
Costo calculado = 54.24
Valor esperado aproximadamente) 54.24


*use la siguiente celda para verificar sus soluciones*

In [ ]:
J = computeCost(X, y, theta=np.array([0.0, 0.0]))
print('Con theta = [0, 0] \nCosto calculado = %.2f' % J)

J = computeCost(X, y, theta=np.array([-1, 2]))
print('With theta = [-1, 2]\nCosto calculado = %.2f' % J)


<a id="section3"></a>
#### 2.2.4 Descenso con Gradiente

Ahora usted implementará la función que determina el descenso con gradiente
La estructura de ciclo se provee como guía. Usted deberá obtener $\theta$ para cada iteración.

Recuerde que la función de costo $J(\theta)$ se parametriza con el vector $\theta$, no con $X$ y tampoco
con $y$. Es decir, minimizamos el valor de $J(\theta)$ cambiando los valores de $\theta$, no cambiando
los valores de $X$ o de $y$. 
[Refiérase a las ecuaciones en este cuaderno](#section2) y las notas y videos de clase. 


El código de inicio de la función ```gradientDescent``` llama a ```computeCost``` en cada iteración y
guarda el costo en una lista. Asumiendo que implementón la función de descenso ```computeCost``` correctamente,  el valor de $J(\theta)$ no debería crecer. Debería converger al final del algoritmo.


<div class="alert alert-box alert-warning">
**Vectors and matrices in `numpy`** - Important implementation notes

La multiplicación de matrices en numpy se hace a través de la instrucción ``` np.dot ```.
Esta instrucción funciona para producto punto, producto de  matriz por vector o producto de matriz por matriz. 
    
Usted puede convertir un vector fila a matriz columna. Por ejemplo  si  `y = np.array([1, 2, 3])` es un vector fila de tamño 3, entonces`y[:,None]` es una matriz de tamaño $3 \times 1$.
<div>
<a id="gradientDescent"></a>

In [16]:
def gradientDescent(X, y, theta, alpha, num_iters):
    """
    Este algoritmo usa descenso en gradiente para aprender "theta".  Las actualizaciones
    de theta se hacen hasta un número de iteraciones `num_iters` con un paso de aprendizaje alpha.
    
    
    Parámetros
    ----------
    X : array
        Los datos de entrada de dimension (m x n+1)
    
    y : array
        Valor con etiquetas, dimension m
    
    theta : array
        Valores iniciales para la regresión. 
        Tamaño n+1 . 
        
    alpha : float
        Paso.
    
    num_iters : int
        Número de iteraciones
    
    Returna
    -------
    theta : array
        El parámetro aprendido de tamaño n+1
    
    J_historia: list
        Lista en Python con los valores de J.
    
    Instrucciones
    ------------
    Ejecuta un paso de gradiente en el vector theta.

    Es conveniente, para hacer debuggging, imprimir los valores de la función de costo.
    
    """
    # Inicialice valroes
    m = y.shape[0]  # número de muestras de entrenamiento
    
    # copie el vector theta
    theta = theta.copy()
    
    
    J_history = [] # Esta lista es conveniente para ver la historia de valores de J.
    theta_history=[]
    for i in range(num_iters):
        # ==================== YOUR CODE HERE =================================
        #Sabiendo que h=X*theta (producto de matriz con vector), se calcula la hipotesis para cada elemento como.
        h = np.dot(X, theta)
        #Derivando, sabemos que el gradiente de la funcion de costo es, es (1/m)*(h_theta(x^(i)-y^i)x^(i)j, para calcular todos los j en una sola iteracion, podemos ver que el x_j en la fila i es 
        #igual al x_i en la columna J, por lo que podemos transponer la matriz X y multiplicar por el vector de errores (h-y). Para obtener el gradiente de todos los j.
        theta = theta - alpha * (1/m) * np.dot(X.T, (h-y))
        theta_history.append(theta)
        #El nuevo theta es el viejo menos el gradiente, multiplicado por alpha.
        # =====================================================================
        
        # Salve el valor de J en cada iteración.
        J_history.append(computeCost(X, y, theta))
        
    return theta, J_history, theta_history

Luego de que acabe con la implementación de `gradientDescent` imprima el valor de $\theta$. 
Inicializamos $\theta$ en 0 y el parámetro de paso en $0.01$. Ejecute la siguiente celda para
verificar su código. 

In [17]:
# inicialice paráemtros
theta = np.zeros(2)

# parámetros de gradiente
iterations = 1500
alpha = 0.01

theta, J_history,theta_history = gradientDescent(X ,y, theta, alpha, iterations)

print('Theta found by gradient descent: {:.4f}, {:.4f}'.format(*theta))
print('Expected theta values (approximately): [-3.6303, 1.1664]')

Theta found by gradient descent: -3.6303, 1.1664
Expected theta values (approximately): [-3.6303, 1.1664]


We will use your final parameters to plot the linear fit. The results should look like the following figure.

![](Figures/regression_result.png)

In [ ]:
# grafique la linea de regresión
plotData(X[:, 1], y)
pyplot.plot(X[:, 1], np.dot(X, theta), '-')
pyplot.legend(['Training data', 'Linear regression']);

Los valores finales de $\theta$ serán usados para hacer predicciones sobre las ganancias en áreas de 35,000 y 70,000 personas.


<div class="alert alert-block alert-success">
Note la forma, en las líneas siguientes, como usamos la multiplicación de matrices en vez de sumas y ciclos para predecir. Este es un ejemplo de vectorización de código en ```numpy``` .

</div>

<div class="alert alert-block alert-success">
Note que el primer argumento de la función de ```numpy```,   ```dot``` es una lista. ```numpy``` puede convertir internamente listas **validas** a arrays cuando se llaman con functiones ```numpy```.
</div>


*muestre sus soluciones en la próxima celda *

In [ ]:
#Predicciones para poblaciones de tamaños   35,000 y 70,000
predict1 = np.dot([1, 3.5], theta)
print('Para una población de tamaño = 35,000, se predice una ganancia de {:.2f}\n'.format(predict1*10000))

predict2 = np.dot([1, 7], theta)
print('Para una población de tamaño = 70,000, se predice una ganancia def {:.2f}\n'.format(predict2*10000))

### 2.4 Visualización de $J(\theta)$

Para entender la función de costo $J(\theta)$ es mejor graficar ésta en una malla 2D, con valores
$\theta_0$ y $\theta_1$. Usted no necesita escribir código para esta parte, pero debe entender
como su código ayuda a crear estas imágenes. 

En la próxima celda, el código está diseñano para calcular $J(\theta)$ sobre una malla de valores
usando la función ```computeCost``` que escribió arriba. Luego de ejecutar la celda, debe tener una
malla 2D con valores de $J(\theta)$. Estos valores producen una superficie y contornos de $J(\theta)$ mediante el uso de ```plot_surface``` y ```contourf``` . Los gráficos deben lucir como las siguientes figuras:

![](Figures/cost_function.png)

El propósito de estas gráficas es mostrar como el $J(\theta)$ varía con cambios en $\theta_0$ y $\theta_1$. El costo $J(\theta)$ es una superficie en forma de taza con un mínimo global. (Esto es más fácil de ver en el gráfico de contorno que en la superfície 3D).  El mínimo es el punto óptimo para $\theta_0$ y $\theta_1$, y cada paso de descenso de gradiente se mueve más cerca a éste punto. 

In [ ]:
from platform import python_version
python_version()

## Ecuaciones normales

<a id="gradientDescentMulti"></a>

<a id="section7"></a>
### 3.1 

En clase se aprendio una forma cerrada de la solución al problema de regresión lineal.

$$ \theta = \left( X^T X\right)^{-1} X^T\vec{y}$$

Inicialmente volvemos a cargar los datos. Aca ayudamos un poco con código para cargar los datos y agregar unos a la matriz $X$. Si observa los datos se dará cuenta que ya son 3D, (tres columnas). O sea que es regresión multivariada.

In [ ]:
# Load data
data = np.loadtxt(os.path.join('Data', 'ex1data2.txt'), delimiter=',')
X = data[:, :2]
y = data[:, 2]
m = y.size
X = np.concatenate([np.ones((m, 1)), X], axis=1)

Complete el código para la función ```normalEqn``` en la celda siguiente para hallar $\theta$.

<a id="normalEqn"></a>

In [ ]:
def normalEqn(X, y):
    """
    Calcula la solución en forma cerrada del problema de regresión lineal usando las ecuaciones normales
    
    Parámetros
    ----------
    X : array
        Los datos de entrada de tamaño m x n+1
    
    y : array
        El valor de las etiquetas. Tamaño m
    
    Retorna
    -------
    theta : array
        Valor estimado del vector de regresion theta de dimensión n+1-
    
    Instrucciones
    ------------
    Complete el código para estimar theta de la solución cerrada.
   
    
    Ayuda
    ----
    No use la pseudoinversa pero puede consultar el mandual de
    `np.linalg.pinv`.
    """
    theta = np.zeros(X.shape[1])
    
    # ===================== YOUR CODE HERE ============================
    #Se programa el metodo de las ecuacionesnormales utilizando np.linalg para invertir la matriz X.T*X
    theta = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)
    # =================================================================
    return theta

*sus soluciones en la próxima celda*

In [ ]:
#se calcula theta con el metodo de las ecuaciones normales y se compara con los valores de referencia y los resultados obtenidos con el gradient descent

theta = normalEqn(X,y)
print('Theta encontrado con el metodo de las ecuaciones normales: {:s}'.format(str(theta)))

Una vez encuentre theta, úselo para predecir el precio de una casa
de 1650 pies cuadrados con 3 dormitorios. 

In [ ]:
# Calcule los parámetros usando ecuaciones normales. 
theta = normalEqn(X, y);

# muestre el resultado
print('Theta de las ecuaciones normales: {:s}'.format(str(theta)));

# Estime el precio  de una casa con   1650 sq-ft y 3 dormitorios
# ====================== YOUR CODE HERE ======================

price = np.dot([1, 1650,3], theta)
#Se estima el precio de una casa de 1650 sq-ft y 3 dormitorios, usando las ecuaciones normales.
# Se usa el producto punto entre el vector theta y el vector de entrada [1,1650,3], que representa la casa.
# ============================================================

print('Precio predicho de una casa de 1650 sq-ft, 3 dormitorios (usando ecuaciones normales): ${:.0f}'.format(price))